# Classifiers workshop

In this workshop we will continue working with calcium imaging data. 

In the calcium workshop you learned about the Peri-Stimulus Time Histogram. Again, PSTH is a way to visualize the average neural response aligned to a specific event like the stimulus onset, reward, or a behavioral cue.

 But when you have hundreds of neurons over several trials, visualising one by one and deciding what is the most likely function is extremely time consuming and inefficient. Luckily there are several approaches that we can use to classify the neurons, using machine learning techniques we learned during the lectures. 

 In this workshop, we will continue doing some exploratory data analysis by using 2 techniques we learned during the lectures: different types of dimensionality reduction and K-means clustering. 

In [ ]:
%pip install umap-learn
%pip install scipy 

In [ ]:
%pip install plotly
%pip install nbformat --upgrade
%pip install jupyterlab "ipywidgets>=7.5"


In [ ]:
#import and load the right libraries

import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.integrate import simpson
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import umap



First, we are going to consolidate what we learned in the calcium data analysis workshop. Let's build a pipeline to sort the neuron responses into its 4 different categories. 

We are going to try 2 approaches to see what gives us better clusters: 

- a raw data dataset, containing the psths for each neuron (so for each neuron we would have an array like [hit_psth, miss_psth, cr_psth, fa_psth]); 
  
- a dataset containing a summary of the features, such as the peak for each psth, the area under the curve, and how long after the stimulus the peak occurs (so for each neuron we would have an array like ['hit_peak', 'miss_peak', 'fa_peak','cr_peak', "hit_latency","miss_latency","fa_latency","cr_latency","hit_auc","miss_auc","fa_auc", "cr_auc"]). 


Since we "don't know what to expect", let's compare and see what the best methods are!

In [ ]:
# we are going to create a dataset with the information we need

data_directory = "S1_trained" # Path to our dataset
output_csv_stats = "neuron_response_features.csv"
output_csv_raw_psths = "neuron_responses_raw.csv"


neuron_features = []

neuron_features_raw= []



for neuron_file in os.listdir(data_directory):
    if not neuron_file.endswith('.mat'):
        continue

    mat = loadmat(os.path.join(data_directory, neuron_file))

    
    # load the variables
    dff = mat['dff'].flatten()
    cues = mat['cues'].flatten()
    reward = mat['reward'].flatten()
    punish =mat['punish'].flatten()
    go= mat['go'].flatten()
    nogo= mat['nogo'].flatten()

    stimtimes = list(np.where(cues ==1)[0])

    # get the number of trials 
    numtrials = len(stimtimes)

    # get the duration of the trial.  The trial duration is the time between one stimulus and the other. 
    #think about it, you have a list of time points where the cues happen. How can you get the amount of time between one cue and the next? 
    trial_duration = stimtimes[1] - stimtimes[0]  # assumes constant duration

    baseline = 2
    post_stim_period = trial_duration - baseline - 1


    # Allocate Containers
    hit_waves = []
    miss_waves = [] 
    fa_waves = [] 
    cr_waves = [] 


    # Sort Trials by Type

    #set the counter to all these to 0
    hit = 0
    miss = 0
    fa = 0
    cr = 0

    for itrial in range(numtrials - 1):
        
        istimstart = stimtimes[itrial] # time of the current trial
        next_stim = stimtimes[itrial + 1] # time of the next trial
        start_idx = istimstart - baseline
        end_idx = istimstart + post_stim_period

        #segment of dff
        segment = dff[start_idx:end_idx]

        # checks if the stimulus is Go
        if go[istimstart] == 1:
            # if the stimulus is Go and there is a reward, it means that the mouse licked for the right stimulus! Hurrah!
            if np.any(reward[istimstart:next_stim] ==1 ):
                hit_waves.append(segment)
                hit+=1
            else:
            #if the stimulus is Go and the mouse did not get a reward, it's a miss
                miss_waves.append(segment)
                miss+=1

        # check if the stimulus is NoGo
        elif nogo[istimstart] ==1:
            # if the mouse was punished, it means it licked, therefore false alarm
            if np.any(punish[istimstart:next_stim] ==1 ):
                fa_waves.append(segment)
                fa+=1
            else:
            # otherwise it correctly rejected the NoGo stimulus
                cr_waves.append(segment)
                cr+=1


    #  Lick-Aligned Waveforms

    # both hit and fa involve the mouse licking (miss and correct rejections means that the mouse doesn't lick)
    hit_lick_waves = []
    fa_lick_waves = []

    #same thing as before 
    for itrial in range(1, numtrials - 1):
        istimstart = stimtimes[itrial]
        next_stim = stimtimes[itrial + 1]
        
    # here it's a bit different, but we basically get whether the mouse was rewarded (aka licked correctly)
        if go[istimstart] == 1:
            licktimes = np.where(reward[istimstart:next_stim] > 0)[0] # the reward is given if the mouse licks correctly 
            if len(licktimes) > 0:
                lick_index = istimstart + licktimes[0]
                start = lick_index - trial_duration // 2
                end = lick_index + trial_duration // 2
                segment = dff[start:end]
                #print(lick_index, licktimes, licktimes[0])

                hit_lick_waves.append(segment)
        #or we get whether the mouse was punished (licked but shouldn't have)
        elif nogo[istimstart] > 0:
            licktimes = np.where(punish[istimstart:next_stim] > 0)[0]
            if len(licktimes) > 0:
                lick_index = istimstart + licktimes[0]
                start = lick_index - trial_duration // 2
                end = lick_index + trial_duration // 2
                segment = dff[start:end]
                fa_lick_waves.append(segment)



    hit_lick_waves = np.array(hit_lick_waves).T if hit_lick_waves else np.empty((trial_duration, 0))
    fa_lick_waves = np.array(fa_lick_waves).T if fa_lick_waves else np.empty((trial_duration, 0))



    # we are now writing a function that takes the waves and computes important information such as psth, the peak of the wave, the area under the curve (auc), and the latency at which the peak occurs. 

    
    def compute_features(waves):
        if len(waves) == 0:
            return np.zeros(post_stim_period + baseline), 0, 0, 0
        psth = np.mean(waves, axis=0)
        peak = np.max(psth)
        auc = simpson(psth)
        latency = np.argmax(psth)
        return psth, peak, auc, latency 

    hit_psth, hit_peak, hit_auc, hit_latency = compute_features(hit_waves)
    miss_psth, miss_peak, miss_auc, miss_latency  = compute_features(miss_waves)
    fa_psth, fa_peak, fa_auc, fa_latency  = compute_features(fa_waves)
    cr_psth, cr_peak, cr_auc, cr_latency  = compute_features(cr_waves)


    # dataset 1: RAW PSTHs
    # Concatenate all PSTHs a
    raw_vector = np.concatenate([hit_psth, miss_psth, fa_psth, cr_psth])
    neuron_features_raw.append({'neuron_id': neuron_file, 'features': raw_vector})


   # dataset 2: features extracted from the data
   #(peak, area under the curve, latency of the peak )
    
    neuron_features.append({
        'neuron_id': neuron_file,
        'hit_peak': hit_peak,
        'hit_auc': hit_auc,
        'hit_latency': hit_latency,
        'miss_peak': miss_peak,
        'miss_auc': miss_auc,
        "miss_latency": miss_latency,
        'fa_peak': fa_peak,
        'fa_auc': fa_auc,
        "fa_latency": fa_latency,
        'cr_peak': cr_peak,
        'cr_auc': cr_auc,
        "cr_latency":cr_latency
        })
    


#store the two different datasets 

raw_df = pd.DataFrame([{
    'neuron_id': entry['neuron_id'],
    **{f'feat_{i}': val for i, val in enumerate(entry['features'])}
} for entry in neuron_features_raw])

#print(raw_df)
raw_df.to_csv(output_csv_raw_psths , index=False)



df = pd.DataFrame(neuron_features)
#print(df)
df.to_csv(output_csv_stats, index=False)
print(f"Feature extraction complete. Saved to {output_csv_stats}")



In [ ]:
# let's check a summary of the first --descriptive-- dataframe
summary_df = pd.read_csv("neuron_response_features.csv")
summary_df[['hit_peak', 'miss_peak', 'fa_peak','cr_peak', "hit_latency","miss_latency","fa_latency","cr_latency","hit_auc","miss_auc","fa_auc", "cr_auc"]].describe()

In [ ]:
#let's check a summary of what the raw data dataset looks like
raw_df = pd.read_csv("neuron_responses_raw.csv")
raw_df.describe()

Now that we have our datasets `neuron_response_features.csv` and `neuron_responses_raw.csv`, let's check the optimal amount of clusters using the elbow method! 

As you remember from the lecture, the elbow method indicates the best number of cluster, which is at the point where there is an "elbow" in the inertia

In [ ]:

 
df = pd.read_csv("neuron_response_features.csv")

# Drop non-numeric columns ( we don't need the neuron_id)
features = df.select_dtypes(include=['float64', 'int64'])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


# Elbow -- we are going to see the inertia score for each k in a range from 2 to 10
inertia = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    inertia.append(kmeans.inertia_)
    
# Plot
plt.figure(figsize=(12, 5))


plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')


plt.tight_layout()
plt.show()



In [ ]:

df = pd.read_csv("neuron_responses_raw.csv")

# Drop non-numeric columns ( we don't need the neuron_id)
features = df.select_dtypes(include=['float64', 'int64'])

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


# Elbow 
inertia = []

k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    inertia.append(kmeans.inertia_)
    

# Plot
plt.figure(figsize=(12, 5))


plt.plot(k_range, inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

plt.tight_layout()
plt.show()

Now that we found the optimal amount of clusters, let's apply dimensionality reduction on our datasets to see if we get some clusters!

Which type of dimensionality reduction? Why not trying them all and compare the results? 

In [ ]:

# Prepare and standardize datasets
feature_data = summary_df.drop(columns=['neuron_id'])
raw_data = raw_df.drop(columns=['neuron_id'])

scaler_feat = StandardScaler()
X_feat = scaler_feat.fit_transform(feature_data)

scaler_raw = StandardScaler()
X_raw = scaler_raw.fit_transform(raw_data)

# Apply dimensionality reduction
pca_feat = PCA(n_components=2).fit_transform(X_feat)
tsne_feat = TSNE(n_components=2, random_state=42).fit_transform(X_feat)
umap_feat = umap.UMAP(random_state=42).fit_transform(X_feat)

pca_raw = PCA(n_components=2).fit_transform(X_raw)
tsne_raw = TSNE(n_components=2, random_state=42).fit_transform(X_raw)
umap_raw = umap.UMAP(random_state=42).fit_transform(X_raw)

# Apply KMeans clustering (default to 4 clusters)
kmeans_feat = KMeans(n_clusters=4, random_state=42).fit(X_feat)
kmeans_raw = KMeans(n_clusters=4, random_state=42).fit(X_raw)

# Add cluster labels to datasets
summary_df['cluster'] = kmeans_feat.labels_
raw_df['cluster'] = kmeans_raw.labels_

# Plotting function
def plot_embeddings(embeddings, labels, title, ax):
    sns.scatterplot(x=embeddings[:,0], y=embeddings[:,1], hue=labels, palette='Set2', s=40, ax=ax)
    ax.set_title(title)
    ax.axis('off')

# Create plots
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
plot_embeddings(pca_feat, kmeans_feat.labels_, "Feature PCA (4 Clusters)", axes[0, 0])
plot_embeddings(pca_raw, kmeans_raw.labels_, "Raw PCA (4 Clusters)", axes[0, 1])
plot_embeddings(tsne_feat, kmeans_feat.labels_, "Feature t-SNE (4 Clusters)", axes[1, 0])
plot_embeddings(tsne_raw, kmeans_raw.labels_, "Raw t-SNE (4 Clusters)", axes[1, 1])
plot_embeddings(umap_feat, kmeans_feat.labels_, "Feature UMAP (4 Clusters)", axes[2, 0])
plot_embeddings(umap_raw, kmeans_raw.labels_, "Raw UMAP (4 Clusters)", axes[2, 1])

plt.tight_layout()
plt.show()


So far PCA seems the clearest, while we might need to increase the dimensionality to get a better understanding of the t-sne and umap clusters. 

But first, in PCA each principal component (PC) is a linear combination of the original features.This allows us to see how much the features contribute to each Component. 

t-SNE and UMAP are nonlinear, and we therefore cannot interpret their dimensions. 

In [ ]:

# Drop non-numerical columns explicitly
feature_names = summary_df.drop(columns=["neuron_id", "preferred_trial"], errors="ignore").select_dtypes(include=np.number).columns

# Prepare the data used for PCA
X_feat = summary_df[feature_names].values

# === 1. Fit PCA model and transform data ===
pca_model_feat = PCA(n_components=3)
pca_feat = pca_model_feat.fit_transform(X_feat)

# === 2. Explained variance per PC ===
print("Explained variance ratio (percent of total variance):")
for i, ratio in enumerate(pca_model_feat.explained_variance_ratio_):
    print(f"PC{i+1}: {ratio:.2%}")

# === 3. Top features in PC1 ===
pc1_weights = pd.Series(pca_model_feat.components_[0], index=feature_names).abs().sort_values(ascending=False)
print("\nTop 5 features contributing to PC1 (most variance):")
print(pc1_weights.head(5))

# === 4. Average contribution across PC1–PC3 ===
avg_contributions = pd.DataFrame(
    np.abs(pca_model_feat.components_[:3]), 
    columns=feature_names
).mean(axis=0).sort_values(ascending=False)

print("\nTop 5 average contributors across PC1–PC3:")
print(avg_contributions.head(5))


In [ ]:
## now do the same for the raw_data pca

Let's give umap and t-sne a fair chance, and increase the dimensions and see if the clusters are clearer now!

In [ ]:
# Load and scale data
features_df = pd.read_csv("neuron_response_features.csv")
raw_df = pd.read_csv("neuron_responses_raw.csv")

X_feat = StandardScaler().fit_transform(features_df.drop(columns=['neuron_id']))
X_raw = StandardScaler().fit_transform(raw_df.drop(columns=['neuron_id']))

# Optional: PCA reduction before t-SNE/UMAP
X_raw_pca50 = PCA(n_components=50).fit_transform(X_raw)

# 3D t-SNE and UMAP
tsne_feat_3d = TSNE(n_components=3, random_state=42).fit_transform(X_feat)
tsne_raw_3d = TSNE(n_components=3, random_state=42).fit_transform(X_raw_pca50)
umap_feat_3d = umap.UMAP(n_components=3, random_state=42).fit_transform(X_feat)
umap_raw_3d = umap.UMAP(n_components=3, random_state=42).fit_transform(X_raw)

# Clustering
kmeans_feat = KMeans(n_clusters=4, random_state=42).fit(X_feat)
kmeans_raw = KMeans(n_clusters=4, random_state=42).fit(X_raw)

# Plotting function
def plot_3d(data, labels, title):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(data[:,0], data[:,1], data[:,2], c=labels, cmap='Set2', s=40)
    ax.set_title(title)
    ax.set_xlabel("Dim 1")
    ax.set_ylabel("Dim 2")
    ax.set_zlabel("Dim 3")
    plt.legend(*scatter.legend_elements(), title="Cluster")
    plt.show()

# Visualize
plot_3d(tsne_feat_3d, kmeans_feat.labels_, "3D t-SNE - Features")
plot_3d(tsne_raw_3d, kmeans_raw.labels_, "3D t-SNE - Raw")
plot_3d(umap_feat_3d, kmeans_feat.labels_, "3D UMAP - Features")
plot_3d(umap_raw_3d, kmeans_raw.labels_, "3D UMAP - Raw")


Fun fact! You can also plot interactive plots!

In [ ]:
import plotly.express as px


# Load data
features_df = pd.read_csv("neuron_response_features.csv")
raw_df = pd.read_csv("neuron_responses_raw.csv")

# Prepare
X_feat = StandardScaler().fit_transform(features_df.drop(columns=["neuron_id"]))
X_raw = StandardScaler().fit_transform(raw_df.drop(columns=["neuron_id"]))
X_raw_pca50 = PCA(n_components=50).fit_transform(X_raw)

# Clustering
kmeans_feat = KMeans(n_clusters=4, random_state=42).fit(X_feat)
kmeans_raw = KMeans(n_clusters=4, random_state=42).fit(X_raw)

# Dimensionality reduction
tsne_feat = TSNE(n_components=3, random_state=42).fit_transform(X_feat)
tsne_raw = TSNE(n_components=3, random_state=42).fit_transform(X_raw_pca50)
umap_feat = umap.UMAP(n_components=3, random_state=42).fit_transform(X_feat)
umap_raw = umap.UMAP(n_components=3, random_state=42).fit_transform(X_raw)

# Create interactive 3D plot
def plot_3d_plotly(embedding, labels, title):
    df = pd.DataFrame(embedding, columns=["x", "y", "z"])
    df["cluster"] = labels
    fig = px.scatter_3d(df, x="x", y="y", z="z", color=df["cluster"].astype(str),
                        title=title, width = 1000, height = 800, labels={"color": "Cluster"})
    fig.update_traces(marker=dict(size=5))
    fig.show()

# Run visualizations
plot_3d_plotly(tsne_feat, kmeans_feat.labels_, "3D t-SNE - Features")
plot_3d_plotly(tsne_raw, kmeans_raw.labels_, "3D t-SNE - Raw")
plot_3d_plotly(umap_feat, kmeans_feat.labels_, "3D UMAP - Features")
plot_3d_plotly(umap_raw, kmeans_raw.labels_, "3D UMAP - Raw")

fig.show()

Now that you have completed this, you can see that there are different clusters emerging from different methods. 
The next step is up to you, but once you identify clusters, it's easier to get an idea of what classifier would be best (eg, would a binary classifier be a good fit? or if there are multiple categories, perhaps a random forest? )